In [1]:
import sys,os
import genome_data_processing as gdp
import ecc_tools as tools
import timeit
# import pydca-ER module
import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
from scipy import linalg
from sklearn.preprocessing import OneHotEncoder
import expectation_reflection as ER
from direct_info import direct_info
from direct_info import sort_di
from joblib import Parallel, delayed
import numpy as np
import pickle
from Bio import Phylo
import networkx, pylab
import pandas as pd
import Bio.SubsMat.FreqTable
from Bio.Align import AlignInfo
from Bio import AlignIO, SeqIO


In [2]:
#========================================================================================
data_path = '/data/cresswellclayec/covid_data/spikeprot1201/'
root_dir = '/data/cresswellclayec/covid_data/'

cpus_per_job = 40


# Swarm aligned file  
msa_file = data_path+"spikeprot1201.fasta"
# ref_file = root_dir+"EPI_ISL_402124.fasta"

In [10]:
all_seqs = []
with open(msa_file, 'r') as f:
    seq_iter = SeqIO.parse(f,'fasta')
#     seq_iter = AlignIO.parse(f,'fasta')
    try:
        for i, seq in enumerate(seq_iter):
            all_seqs.append(seq)
    except(UnicodeDecodeError):
        print('error at i = %d'%i)
print(len(all_seqs))


error at i = 11678610
11678611


In [11]:
print(type(all_seqs[0]))

<class 'Bio.SeqRecord.SeqRecord'>


In [12]:
variant_metadata = pd.read_csv("/data/cresswellclayec/covid_data/variant_surveillance/variant_surveillance.tsv",sep='\t',low_memory=False)
print(variant_metadata.head())
print(len(variant_metadata))
variant_metadata = variant_metadata.dropna(subset=['Accession ID', 'Variant'])
print(len(variant_metadata))
variant_metadata['Collection date'] = pd.to_datetime(variant_metadata['Collection date'])

omicron_df = variant_metadata[variant_metadata['Variant'].str.contains('Omicron')]
delta_df = variant_metadata[variant_metadata['Variant'].str.contains('Delta')]



       Accession ID             Type Clade Pango lineage Pangolin version  \
0  EPI_ISL_10588695  betacoronavirus   GRA          BA.1      PANGO-v1.16   
1  EPI_ISL_13380166  betacoronavirus   GRA     BA.2.12.1   consensus call   
2  EPI_ISL_10534395  betacoronavirus   GRA        BA.1.1     PLEARN-v1.16   
3  EPI_ISL_10535295  betacoronavirus   GRA          BA.2   consensus call   
4   EPI_ISL_9513747  betacoronavirus   GRA        BA.1.1      PANGO-v1.16   

                                    AA Substitutions  \
0  (NSP5_P132H,Spike_H69del,Spike_A67V,Spike_S373...   
1  (NSP5_P132H,NSP3_G489S,Spike_L24del,NSP4_T327I...   
2  (NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...   
3  (NSP5_P132H,NSP3_G489S,Spike_L24del,NSP4_T327I...   
4  (NSP5_P132H,Spike_T95I,Spike_A67V,Spike_S373P,...   

                                             Variant Collection date  \
0  VOC Omicron GRA (B.1.1.529+BA.*) first detecte...      2022-01-27   
1  VOC Omicron GRA (B.1.1.529+BA.*) first detecte...    

In [13]:
print(len(omicron_df))
print(len(delta_df))

print(omicron_df.head)

6679236
4550038
<bound method NDFrame.head of               Accession ID             Type Clade Pango lineage  \
0         EPI_ISL_10588695  betacoronavirus   GRA          BA.1   
1         EPI_ISL_13380166  betacoronavirus   GRA     BA.2.12.1   
2         EPI_ISL_10534395  betacoronavirus   GRA        BA.1.1   
3         EPI_ISL_10535295  betacoronavirus   GRA          BA.2   
4          EPI_ISL_9513747  betacoronavirus   GRA        BA.1.1   
...                    ...              ...   ...           ...   
14050354  EPI_ISL_10533295  betacoronavirus   GRA     BA.1.15.1   
14050355  EPI_ISL_10588595  betacoronavirus   GRA        BA.2.9   
14050356  EPI_ISL_10534095  betacoronavirus   GRA        BA.1.1   
14050357   EPI_ISL_8901256  betacoronavirus   GRA       BA.1.17   
14050358  EPI_ISL_13380115  betacoronavirus   GRA          BA.2   

         Pangolin version                                   AA Substitutions  \
0             PANGO-v1.16  (NSP5_P132H,Spike_H69del,Spike_A67V,Spike_

In [14]:
print(variant_metadata.groupby([pd.PeriodIndex(variant_metadata['Collection date'], freq='Y'), 
                                variant_metadata['Variant']]).size())

Collection date  Variant                                                                           
2017             VOC Delta GK (B.1.617.2+AY.*) first detected in India                                       1
                 VOC Gamma GR/501Y.V3 (P.1+P.1.*) first detected in Brazil/Japan                             4
2019             VOC Alpha GRY (B.1.1.7+Q.*) first detected in the UK                                        6
                 VOC Delta GK (B.1.617.2+AY.*) first detected in India                                       1
2020             VOC Alpha GRY (B.1.1.7+Q.*) first detected in the UK                                    22791
                 VOC Beta GH/501Y.V2 (B.1.351+B.1.351.2+B.1.351.3) first detected in South Africa         3225
                 VOC Delta GK (B.1.617.2+AY.*) first detected in India                                     742
                 VOC Gamma GR/501Y.V3 (P.1+P.1.*) first detected in Brazil/Japan                           198
            

In [15]:
# get sequences fro omicron
print(all_seqs[0].id)
print(all_seqs[0].id.split('|'))
print(all_seqs[0].id.split('|')[3])
accessions = []
accessions_indx = []
for i,seq in enumerate(all_seqs):
    if 'EPI_ISL' in seq.id:
        for id_str in seq.id.split('|'):
            if 'EPI_ISL' in id_str:
                accessions.append(id_str)
                accessions_indx.append(i)
                break
    

Spike|hCoV-19/Wuhan/WIV04/2019|2019-12-30|EPI_ISL_402124|Original|hCoV-19^^Hubei|Human|Wuhan
['Spike', 'hCoV-19/Wuhan/WIV04/2019', '2019-12-30', 'EPI_ISL_402124', 'Original', 'hCoV-19^^Hubei', 'Human', 'Wuhan']
EPI_ISL_402124


In [16]:
print(len(accessions))

11399377


In [17]:
omicron_prot_df = omicron_df.loc[omicron_df['Accession ID'].isin(accessions)]
print(len(omicron_prot_df))
delta_prot_df = delta_df.loc[delta_df['Accession ID'].isin(accessions)]
print(len(delta_prot_df))

4367191
4339785


In [18]:
# # Make sure there are no duplicates -- passed
# print(len(delta_prot_df['Accession ID'].unique()))
# print(len(omicron_prot_df['Accession ID'].unique()))
# print(len(pd.concat([delta_prot_df,omicron_prot_df])['Accession ID'].unique()))
# print(len(omicron_prot_df) + len(delta_prot_df))

In [19]:
full_df = pd.concat([delta_prot_df,omicron_prot_df])
print(full_df.groupby([pd.PeriodIndex(full_df['Collection date'], freq='Y'), 
                                full_df['Variant']]).size())

Collection date  Variant                                                                           
2017             VOC Delta GK (B.1.617.2+AY.*) first detected in India                                       1
2019             VOC Delta GK (B.1.617.2+AY.*) first detected in India                                       1
2020             VOC Delta GK (B.1.617.2+AY.*) first detected in India                                     508
2021             VOC Delta GK (B.1.617.2+AY.*) first detected in India                                 4290277
                 VOC Omicron GRA (B.1.1.529+BA.*) first detected in Botswana/Hong Kong/South Africa     476843
2022             VOC Delta GK (B.1.617.2+AY.*) first detected in India                                   48998
                 VOC Omicron GRA (B.1.1.529+BA.*) first detected in Botswana/Hong Kong/South Africa    3890348
dtype: int64


In [22]:
# omicron_prot_df.to_pickle("./omicron_prot_df.pkl")
# delta_prot_df.to_pickle("./delta_prot_df.pkl")
# np.save('accessions.npy', accessions)
# np.save('accessions_indx.npy', accessions_indx)
delta_prot_df = pd.read_pickle("./delta_prot_df.pkl")
omicron_prot_df = pd.read_pickle("./omicron_prot_df.pkl")

accessions = np.load('accessions.npy').tolist()
accessions_indx = np.load('accessions_indx.npy').tolist()

# get the msa indices of the variant data
print(accessions[:5])

['EPI_ISL_402124', 'EPI_ISL_2155626', 'EPI_ISL_940774', 'EPI_ISL_1097311', 'EPI_ISL_960831']


In [23]:
# delta_prot_df['MSA indx'] = pd.Series(dtype='int')

def get_acc_indx(accessions, accessions_indx, acc_id, i):
    indx = accessions_indx[accessions.index(acc_id[i])]
    return indx
def get_acc_indx(accessions, acc_id):
    return acc_id

In [24]:
# print(get_acc_indx(accessions, accessions_indx, delta_prot_df['Accession ID'].tolist(),0))
delta_accession = delta_prot_df['Accession ID'].tolist()
omicron_accession = omicron_prot_df['Accession ID'].tolist()
accession_as_dict = dict(zip(accessions,range(0,len(accessions))))


In [25]:
msa_indx = accession_as_dict[delta_accession[0]]
print(msa_indx)
print(delta_accession[0])
print(accessions[msa_indx])


7233731
EPI_ISL_8997419
EPI_ISL_8997419


In [26]:
delta_indx = []
for acc in delta_accession:
    delta_indx.append(accession_as_dict[acc])
omicron_indx = []
for acc in omicron_accession:
    omicron_indx.append(accession_as_dict[acc])

In [27]:
print(len(delta_indx))
print(len(omicron_indx))
print(len(delta_prot_df))
print(len(omicron_prot_df))



4339785
4367191
4339785
4367191


In [47]:
# create MSA of omicron and delta variants
omicron_seqs = [all_seqs[accessions_indx[i]] for i in omicron_indx] 
print(len(np.array(list(omicron_seqs[0].seq))))
omicron_char_mat = np.chararray((len(omicron_seqs),len(omicron_seqs[0].seq)))
for i,seq in enumerate(omicron_seqs):
    omicron_char_mat[i,:] = np.array(list(omicron_seqs[0].seq))

1271


In [50]:
print(omicron_char_mat.shape)
np.save('omicron_s.npy', omicron_char_mat)

(4367191, 1271)


In [51]:
delta_seqs = [all_seqs[accessions_indx[i]] for i in delta_indx]
delta_char_mat = np.chararray((len(delta_seqs),len(delta_seqs[0].seq)))
for i,seq in enumerate(delta_seqs):
    delta_char_mat[i,:] = np.array(list(delta_seqs[0].seq))

In [52]:
np.save('delta_s.npy', delta_char_mat)

# accessions_indx gives the indx of accession in original sequence set
output_handle = open("omicron.fa", "w")
AlignIO.write(omicron_seqs, output_handle, "fasta")
output_handle.close()

# create MSA of omicron and delta variants
output_handle = open("delta.fa", "w")


AlignIO.write(delta_seqs, ouptut_handle, "fasta")
output_handle.close()

Exception ignored in: <_io.FileIO name='omicron.fa' mode='wb' closefd=True>
Traceback (most recent call last):
  File "/tmp/ipykernel_77663/1903669058.py", line 4, in <module>


TypeError: Expect a list or iterator of MultipleSeqAlignment objects, got: SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...YT*'), id='Spike|hCoV-19/Spain/MC-HCUVA-16186079/2022|2022-01-27|EPI_ISL_10588695|Original|hCoV-19^^Murcia|Human|Hospital', name='Spike|hCoV-19/Spain/MC-HCUVA-16186079/2022|2022-01-27|EPI_ISL_10588695|Original|hCoV-19^^Murcia|Human|Hospital', description='Spike|hCoV-19/Spain/MC-HCUVA-16186079/2022|2022-01-27|EPI_ISL_10588695|Original|hCoV-19^^Murcia|Human|Hospital Cl\\u00ednico Universitario Virgen de la Arrixaca|Hospital Cl\\u00ednico Universitario Virgen de la Arrixaca|Sim\\u00f3n P\\u00e1ez|Spain', dbxrefs=[])